## Abstract

The notebook contains as simple class (SU) representating arbitrary 3D rotation and reflection operators in the basis of the common eigenvectors of $\hat{L}^2$ and $\hat{L}_z$. 

In [32]:
import numpy as np

class SU:
    # Class represents the group of all rotations in 
    # 3D space on a space of angular momentum functions
    
    def __init__(self, L):
        # Do some sanity checks
        assert L >= 0
        self.L = int(L)
        
        # Calculate the generator matrices (L_x, L_y, L_z)
        self.Lz = np.diag([m for m in range(-L, L + 1)])
        Lp = np.diag([ np.sqrt(L*(L + 1) - m * (m + 1)) for m in range(-L, L)], k = -1)
        Lm = np.diag([ np.sqrt(L*(L + 1) - m * (m - 1)) for m in range(-L + 1, L + 1)], k = +1)
        self.Lx = 0.5 * (Lp + Lm)
        self.Ly = -0.5j * (Lp - Lm)
        
               
        # Make sure  that everything is correct and basic properties 
        # of Lx, Ly and Lz are satisfied
        # 1. Is the shape correct?
        assert self.Lx.shape == (2*L + 1, 2*L + 1) and self.Ly.shape == (2*L + 1, 2*L + 1) and self.Lz.shape == (2*L + 1, 2*L + 1)
        # 2. Are they hermitian?
                
        assert np.allclose(self.Lx, np.conj(self.Lx.T)) and np.allclose(self.Ly, np.conj(self.Ly.T)) and np.allclose(self.Lz, np.conj(self.Lz.T))
        
        # 3. Diagonalize and check that the eigenvalues are correct
        m_ref = np.array([m for m in range(-L, L + 1)], dtype=np.float64)
        zero = np.zeros(2*L + 1)
        
        mx = np.linalg.eigvalsh(self.Lx)
        mx_re = np.sort(np.real(mx))
        my = np.linalg.eigvalsh(self.Ly)
        my_re = np.sort(np.real(my))
        mz = np.linalg.eigvalsh(self.Lz)
        mz_re = np.sort(np.real(mz))
        
        assert np.allclose(mx_re, m_ref) and np.allclose(my_re, m_ref) and np.allclose(mz_re, m_ref)
        assert np.allclose(zero, np.imag(mx)) and np.allclose(zero, np.imag(my)) and np.allclose(zero, np.imag(mz))
        
        # Reference reflection matrix (corresponds to XY plane: theta -> pi - theta)
        self.SigmaXY = np.diag([(-1)**(L + m) for m in range(-L, L + 1)])
        
        
    def gen_op(self, ang, ax, op_type = 'Cn'):
        # Returns the matrix of the operator
        # represented in the basis of angular
        # momentum functions
        # ang (double)   - rotation angle in rad; relevant for rotation operators
        # ax (double[3]) - direction of the rotation axis (for rotation operators)/normal vector to the mirror plane 
        # op_typ (char)    - 'rot' (rotation), 'ref' (reflection)
        
        ax = np.asarray(ax)
        ax = ax / np.sqrt(np.sum(ax**2)) # normalize
        ang = np.abs(ang)
        
        assert len(ax) == 3 and ang < 2. * np.pi
        
        if op_type == 'Cn':
            return self._rotation(ang, ax)
        elif op_type == 'Sigma':
            return self._reflection(ax)
        elif op_type == 'Sn':
            return np.dot(self._reflection(ax), self._rotation(ang, ax))
        else:
            print("Operator type is not recognized; please specifiy either rotation (rot) or reflection (ref).")
            
    def _rotation(self, ang, ax):
        # The ax (axis) vector is assumed to be normalized
        
        M = ax[0] * self.Lx + ax[1] * self.Ly + ax[2] * self.Lz
        e, v = np.linalg.eigh(M)
        R = np.einsum("ij,jk,kl->il", v, np.diag(np.exp(-1.j * ang * e)), np.conj(v.T))
            
        return R
                    
    def _reflection(self, ax):
        # The normal vector to the plane is assumed to be normalized
        
        if ax[2] < 0:
            ax = -ax
        
        ax_XY = np.array([0., 0., 1.])
        ang = np.dot(ax_XY, ax) / (np.sqrt(np.sum(ax**2)))
        rot_ax = np.cross(ax_XY, ax)
        rot_ax = rot_ax / np.sqrt(np.dot(rot_ax, rot_ax)) # positive rotation angle corresponds to the rotation 
                                                          # __from__ the XY plane to the plane defined by ax
        
        return np.einsum("ij, jk, kl-> il", self._rotation(ang, rot_ax), self.SigmaXY, self._rotation(-ang, rot_ax))
        
       

In [49]:
L = 2 # Will look at D-term

# Create a generator
D = SU(L)

# This will represent an instance of the Abelian D2 group
E = np.eye(2*L + 1)
Cz = D.gen_op(np.pi, [0., 0,  1.])
Cx = D.gen_op(np.pi, [1., 0, 0])
Cy = D.gen_op(np.pi, [0., 1., 0])

D2_group = [E, Cz, Cy, Cx]

# Character table of the D2 can be found on-line at https://www.webqc.org/symmetrypointgroup-d2.html
# It is not hard to see that D term will have the following splitting in D2 field
# D = 2A + B1 + B2 + B3
# Below, I will verify that explicitly and will assign the basis functions to the symmetry types

character_table_D2 = {'A': np.ones(len(D2_group)), 'B1' : np.array([1, 1, -1, -1]), 'B2' : np.array([1, -1, 1, -1]), 'B3' : np.array([1, -1, -1, 1]) }

# First calculate the character of D representation in group D2

chi = []
for op in D2_group:
    chi.append(np.real(np.einsum("ii->", op)))

chi = np.array(chi)
#print(chi)
# Perform dot-product of chi with the characters of irreducible representations 

thresh = 1e-6

for ichar in character_table_D2:
    ch = character_table_D2[ichar]
    #print(ch)
    dp = ch[0] / len(D2_group) * np.dot(ch, chi)
    print(" chi * %s = %s" % (ichar, dp))
    
proj_operators = {}

for ichar in character_table_D2:
    ch = character_table_D2[ichar]
    P = np.zeros((2*L + 1, 2*L + 1), dtype=np.complex128)
    for i in range(len(D2_group)):
        P += (ch[i] * D2_group[i])
    proj_operators[ichar] = P
    
 

 chi * A = 1.9999999999999996
 chi * B1 = 1.0000000000000004
 chi * B2 = 1.0
 chi * B3 = 1.0


In [50]:
# After the projection operators are calculated we can explore the structure of the 
# symmetry types in a bit more detail

# E.g. A 
print(np.real(proj_operators['A']))

[[ 2.00000000e+00  1.11022302e-16  0.00000000e+00  1.14942098e-15
   2.00000000e+00]
 [ 3.33066907e-16  1.11022302e-16 -1.56523910e-15  0.00000000e+00
   1.35758779e-15]
 [ 0.00000000e+00  1.45421679e-15  4.00000000e+00 -1.39870564e-15
   0.00000000e+00]
 [-1.09390983e-15  0.00000000e+00  1.62075025e-15 -1.77635684e-15
  -2.22044605e-16]
 [ 2.00000000e+00 -8.85743009e-16  0.00000000e+00 -4.93038066e-31
   2.00000000e+00]]


There are two functions of A type $| A; 1 \rangle = | 2, -2 \rangle + | 2, -2 \rangle$ and $| A; 2 \rangle = | 2, 0\rangle$

In [51]:
print(np.real(proj_operators['A'])[:,0])

[ 2.00000000e+00  3.33066907e-16  0.00000000e+00 -1.09390983e-15
  2.00000000e+00]
